In [1]:
import numpy as np
import astropy.units as u

from einsteinpy.utils import kerr_utils
from einsteinpy.coordinates import BoyerLindquist
from initialcondition import InitialConditionsBHFrame

from constants import MotionConstants

In [2]:
inicons = MotionConstants(3060 * u.m, np.pi/2 * u.rad, -np.pi/6*u.rad , 0.2 * u.km , 10 * u.kg , 0 * u.km ,0 * u.km , 0 * u.km)

In [3]:
inicons.carter_const()

TypeError: 'BoyerLindquistDifferential' object is not subscriptable